In [7]:
search_for = 120
start_from = 133819999
threads = 5
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0024895668029785156
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 875
all cases: 1116151311
type: [1, 1, 1, 1, 120] 120
cases of this type: 207360000
50000 0.21496239706039297
100000 0.20049830988015202
150000 0.20048489280317627
200000 0.15802547495590183
250000 0.18584148027627073
300000 0.20229361976063254
350000 0.16846060306896635
400000 0.15854396408500804
450000 0.21715377892862955
500000 0.2167332373448557
550000 0.21269244816928173
600000 0.20135119437993765
650000 0.20551472750097247
700000 0.14424671986929707
750000 0.1655015490596056
800000 0.1756921803430385
850000 0.20758151255420446
900000 0.2003028879779657
950000 0.1966350387454152
1000000 0.19690391198194954
1050000 0.20430078564594983
1100000 0.20001884161910483
1150000 0.14547192935612466
1200000 0.1566152924054861
1250000 0.24541202435034512
1300000 0.2042741792147345
1350000 0.19838516677150725
1400000 0.19653797514342866
1450000 0.19

14600000 0.20330795020177894
14650000 0.19276626601914962
14700000 0.194143494379127
14750000 0.19950473955537742
14800000 0.19863499523666697
14850000 0.18570604059362011
14900000 0.19837773945066664
14950000 0.20546187330801358
15000000 0.2133825577620665
15050000 0.1849095405655755
15100000 0.19861734839780995
15150000 0.17853297590752837
15200000 0.19825807609426313
15250000 0.19817032955250605
15300000 0.20438317002223333
15350000 0.19722937443037564
15400000 0.20134654912217193
15450000 0.19896680688168208
15500000 0.19903357768451374
15550000 0.19957266844100424
15600000 0.19995427426662446
15650000 0.20267248389473094
15700000 0.2008937490497364
15750000 0.2075382638948361
15800000 0.2013885221217036
15850000 0.18160826188328666
15900000 0.19768024693673847
15950000 0.198733612678202
16000000 0.20463208132466737
16050000 0.19823469420762063
16100000 0.20302527423980235
16150000 0.20497863085804383
16200000 0.19688330810862778
16250000 0.19756054504641693
16300000 0.201607310678

28950000 0.1953168553955714
29000000 0.1892153016947521
29050000 0.19691303780765404
29100000 0.19670240183624027
29150000 0.19729806403353875
29200000 0.19514668709635072
29250000 0.18654154469107787
29300000 0.1952568567013396
29350000 0.18942213602934546
29400000 0.19003382600095273
29450000 0.19818641094230016
29500000 0.19543106689397496
29550000 0.21216636983520984
29600000 0.20153107461749314
29650000 0.21870780486080776
29700000 0.2055706662968278
29750000 0.22120303038521738
29800000 0.2205942574247453
29850000 0.18995801485673586
29900000 0.20046396077872386
29950000 0.19724637579318946
30000000 0.2053038581692576
30050000 0.2380914493063198
30100000 0.23543503782948388
30150000 0.20004094748434226
30200000 0.1973779000795298
30250000 0.2085474978365143
30300000 0.1959876227326353
30350000 0.21246894873243305
30400000 0.22518229521165956
30450000 0.2068164875014305
30500000 0.221057512276419
30550000 0.1957009273446785
30600000 0.21838429062365292
30650000 0.20608105713386135

43500000 0.21708490022238494
43550000 0.23194963115823666
43600000 0.21808468540896445
43650000 0.195299794587783
43700000 0.1966572231003324
43750000 0.19631282910605932
43800000 0.21176593741880656
43850000 0.19114730455569556
43900000 0.19429418259028988
43950000 0.1931617587297916
44000000 0.19478304361867904
44050000 0.19514033258623284
44100000 0.2349631764886896
44150000 0.19397625803552868
44200000 0.1928682462802291
44250000 0.19284079295431772
44300000 0.1919558315891385
44350000 0.1885595877016001
44400000 0.18425654459338187
44450000 0.1965707970239679
44500000 0.18516625545904236
44550000 0.19368671271064283
44600000 0.1847245999530289
44650000 0.18408157367260322
44700000 0.18395519115106265
44750000 0.18334773723096057
44800000 0.21069999708860315
44850000 0.18616718464330437
44900000 0.1869048495277471
44950000 0.18688051842905679
45000000 0.18684767604158323
45050000 0.18653534713267086
45100000 0.19912290839613808
45150000 0.2019451229157925
45200000 0.193547950488690

57850000 0.19254439602472784
57900000 0.20406291532251836
57950000 0.21488092791435054
58000000 0.2128118657530917
58050000 0.21467982744688194
58100000 0.21341259173603988
58150000 0.21401350526278154
58200000 0.21274922150855857
58250000 0.21201612652959556
58300000 0.21001783934150403
58350000 0.21311837303940057
58400000 0.20508873483619824
58450000 0.19034637877893051
58500000 0.19228604587612946
58550000 0.1991284804308931
58600000 0.20883065402044457
58650000 0.2128562287800749
58700000 0.21247219254368677
58750000 0.2249174901275886
58800000 0.2122196146591902
58850000 0.21219697521001893
58900000 0.22094771609789396
58950000 0.21745188657782874
59000000 0.21486374698614677
59050000 0.22732359719909961
59100000 0.22055729087088902
59150000 0.18456000860699548
59200000 0.19348876581018234
59250000 0.18937365855842828
59300000 0.19704427638010313
59350000 0.18660376597523554
59400000 0.18576210536963939
59450000 0.19297691617293491
59500000 0.19160738842995034
59550000 0.21161423

...

best so far: 0
type: [2, 3, 4, 1, 5] 120
cases of this type: 30000
best so far: 0
type: [2, 3, 4, 5, 1] 120
cases of this type: 6000
best so far: 0
type: [2, 3, 5, 1, 4] 120
cases of this type: 19200
1101700000 0.21445807073715925
best so far: 0
type: [2, 3, 5, 2, 2] 120
cases of this type: 9600
best so far: 0
type: [2, 3, 5, 4, 1] 120
cases of this type: 4800
best so far: 0
type: [2, 3, 10, 1, 2] 120
cases of this type: 4800
best so far: 0
type: [2, 3, 10, 2, 1] 120
cases of this type: 2400
best so far: 0
type: [2, 3, 20, 1, 1] 120
cases of this type: 1200
best so far: 0
type: [2, 4, 1, 1, 15] 120
cases of this type: 202500
1101750000 0.258948247921745
1101800000 0.43994884736696216
1101850000 0.22358707933921945
1101900000 0.28969490949795246
best so far: 0
type: [2, 4, 1, 3, 5] 120
cases of this type: 67500
1101950000 0.3519288475854079
1102000000 0.17046898888538678
best so far: 0
type: [2, 4, 1, 5, 3] 120
cases of this type: 40500
best so far: 0
type: [2, 4, 1, 15, 1] 120
cases of

1107200000 0.23910154195093974
1107250000 0.15457752596595156
1107300000 0.1429359553170562
1107350000 0.15072412865922982
1107400000 0.19005048232448896
1107450000 0.24927093029464487
1107500000 0.25675928307703466
1107550000 0.21268402579186757
1107600000 0.215984250646619
best so far: 0
type: [3, 1, 1, 8, 5] 120
cases of this type: 320000
1107650000 0.1738233659305533
1107700000 0.22712731967219643
1107750000 0.19254503308092358
1107800000 0.19874509223429623
1107850000 0.18126697633605135
1107900000 0.20384082976592385
best so far: 0
type: [3, 1, 1, 10, 4] 120
cases of this type: 256000
1107950000 0.23558246476549707
1108000000 0.21256717018348512
1108050000 0.1861918728331089
1108100000 0.2415793575531178
1108150000 0.2525457067087995
best so far: 0
type: [3, 1, 1, 20, 2] 120
cases of this type: 128000
1108200000 0.14788584542752503
1108250000 0.10805759380632506
1108300000 0.09056224761736524
best so far: 0
type: [3, 1, 1, 40, 1] 120
cases of this type: 64000
1108350000 0.0898578

best so far: 0
type: [4, 1, 1, 2, 15] 120
cases of this type: 405000
1112150000 0.18174273732939564
1112200000 0.1102159962085578
1112250000 0.1151254604955554
1112300000 0.10649526339902878
1112350000 0.047317962491545414
1112400000 0.04685491657983463
1112450000 0.09588224024352102
1112500000 0.10050850562652615
best so far: 0
type: [4, 1, 1, 3, 10] 120
cases of this type: 270000
1112550000 0.09467548301784198
1112600000 0.08221921038798095
1112650000 0.0626973671019223
1112700000 0.0920063465309143
1112750000 0.057449764567763274
best so far: 0
type: [4, 1, 1, 5, 6] 120
cases of this type: 162000
1112800000 0.08055412484683726
1112850000 0.06742637517736355
1112900000 0.06792169164757067
1112950000 0.06803028080307642
best so far: 0
type: [4, 1, 1, 6, 5] 120
cases of this type: 135000
1113000000 0.05599693578018744
1113050000 0.05991401928817696
best so far: 0
type: [4, 1, 1, 10, 3] 120
cases of this type: 81000
1113100000 0.05400228963054418
1113150000 0.05798755636081696
best so f

1115300000 0.013527269432707628
best so far: 0
type: [5, 2, 1, 2, 6] 120
cases of this type: 20736
best so far: 0
type: [5, 2, 1, 3, 4] 120
cases of this type: 13824
1115350000 0.013730462045656311
best so far: 0
type: [5, 2, 1, 4, 3] 120
cases of this type: 10368
best so far: 0
type: [5, 2, 1, 6, 2] 120
cases of this type: 6912
best so far: 0
type: [5, 2, 1, 12, 1] 120
cases of this type: 3456
best so far: 0
type: [5, 2, 2, 1, 6] 120
cases of this type: 10368
best so far: 0
type: [5, 2, 2, 2, 3] 120
cases of this type: 5184
best so far: 0
type: [5, 2, 2, 3, 2] 120
cases of this type: 3456
best so far: 0
type: [5, 2, 2, 6, 1] 120
cases of this type: 1728
best so far: 0
type: [5, 2, 3, 1, 4] 120
cases of this type: 4608
1115400000 0.012185808683967592
best so far: 0
type: [5, 2, 3, 2, 2] 120
cases of this type: 2304
best so far: 0
type: [5, 2, 3, 4, 1] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 4, 1, 3] 120
cases of this type: 2592
best so far: 0
type: [5, 2, 4, 3, 1] 120


best so far: 0
type: [10, 1, 3, 4, 1] 120
cases of this type: 576
best so far: 0
type: [10, 1, 4, 1, 3] 120
cases of this type: 1296
1116100000 0.000556664425444603
best so far: 0
type: [10, 1, 4, 3, 1] 120
cases of this type: 432
best so far: 0
type: [10, 1, 6, 1, 2] 120
cases of this type: 576
best so far: 0
type: [10, 1, 6, 2, 1] 120
cases of this type: 288
best so far: 0
type: [10, 1, 12, 1, 1] 120
cases of this type: 144
best so far: 0
type: [10, 2, 1, 1, 6] 120
cases of this type: 2592
best so far: 0
type: [10, 2, 1, 2, 3] 120
cases of this type: 1296
best so far: 0
type: [10, 2, 1, 3, 2] 120
cases of this type: 864
best so far: 0
type: [10, 2, 1, 6, 1] 120
cases of this type: 432
best so far: 0
type: [10, 2, 2, 1, 3] 120
cases of this type: 648
best so far: 0
type: [10, 2, 2, 3, 1] 120
cases of this type: 216
best so far: 0
type: [10, 2, 3, 1, 2] 120
cases of this type: 288
best so far: 0
type: [10, 2, 3, 2, 1] 120
cases of this type: 144
best so far: 0
type: [10, 2, 6, 1, 1] 12